In [ ]:
import pandas as pd
import model
import numpy as np
import os
import pickle
import torch


In [ ]:
# without description_foul_bunt and description_foul_tip + other stuff for live data model integration
hasDataSet = os.path.exists('./data/train_dataset_f_emb.pkl')
print(hasDataSet)
train_dataset, val_dataset = model.load_training_data('./data/final_final_encoded_savant_2024.csv', hasDataSet, './data/batting_stats_23.csv', './data/pitching_stats_23.csv')

False
reading batter csv
read batter csv
reading pitcher csv
read pitcher csv
reading dataset csv
read dataset csv
processing train dataset


Processing plate appearances: 100%|██████████| 179716/179716 [02:33<00:00, 1173.10it/s]


processing val dataset


Processing plate appearances: 100%|██████████| 100815/100815 [00:23<00:00, 4266.68it/s]


finished processing datasets f


In [ ]:
df = pd.read_csv('./data/final_final_encoded_savant_2024.csv')
# List of memory features (copy from your model)
memory_features = [
    'release_speed', 'release_pos_y',
    'pitch_type_group_BREAK', 'pitch_type_group_FAST', 'pitch_type_group_OFF',
    'description_ball', 'description_blocked_ball', 'description_called_strike', 'description_foul', 
    'description_foul_bunt', 'description_foul_tip', 'description_hit_by_pitch', 'description_hit_into_play', 
    'description_pitchout', 'description_swinging_strike', 'description_swinging_strike_blocked', 
    'type_B', 'type_S', 'bb_type_fly_ball', 'bb_type_ground_ball', 'bb_type_line_drive', 'bb_type_popup', 
    'zone_1.0', 'zone_2.0', 'zone_3.0', 'zone_4.0', 'zone_5.0', 'zone_6.0', 'zone_7.0', 'zone_8.0', 
    'zone_9.0', 'zone_11.0', 'zone_12.0', 'zone_13.0', 'zone_14.0', 'zone_nan', 'hit_location_1.0', 'hit_location_2.0', 
    'hit_location_3.0', 'hit_location_4.0', 'hit_location_5.0', 'hit_location_6.0', 'hit_location_7.0', 'hit_location_8.0', 
    'hit_location_9.0', 'hit_location_nan'
]

# Number of previous pitches to mock
num_prev_pitches = 5

# Create random data for each feature
mock_data = {}
np.random.seed(42)  # For reproducibility

# Example: assign plausible ranges for numeric features, 0/1 for one-hot/categorical
for feature in memory_features:
    if feature == 'release_speed':
        mock_data[feature] = np.random.uniform(-0.75, 100, num_prev_pitches)  # plausible pitch speeds
    elif feature == 'release_pos_y':
        mock_data[feature] = np.random.uniform(-0.7, 0.85, num_prev_pitches)   # plausible y positions
    else:
        # For one-hot/categorical features, use 0 or 1
        mock_data[feature] = np.random.randint(0, 2, num_prev_pitches).astype(float)

mock_memory = pd.DataFrame(mock_data).to_numpy()
context_features = [
    'stand_R', 'p_throws_R', 'balls', 'strikes',
    'on_3b', 'on_2b', 'on_1b', 'outs_when_up', 'inning', 'inning_topbot_Top', 'at_bat_number', 'pitch_number',
    'home_score', 'away_score', 'bat_score', 'fld_score', 'n_thruorder_pitcher', 'n_priorpa_thisgame_player_at_bat',
    'prev_runs_scored'
]
context = df.iloc[100][context_features].astype(float).values

In [ ]:
# Load model config/specs
with open("best_pitch_predictor_lstm_meta.pkl", "rb") as f:
    meta = pickle.load(f)

for key in meta:
    print(key)
    if (key == 'lstm_init_args'):
        for k in meta[key]:
            print('   ', k)

In [ ]:
import pickle
import torch
import model

# Load model config/specs
with open("best_pitch_predictor_lstm_meta.pkl", "rb") as f:
    meta = pickle.load(f)

# Instantiate the model using the loaded specs
loaded_mod = model.PitchPredictorLSTM(
    context_dim=meta["lstm_init_args"]['context_dim'],
    memory_dim=meta["lstm_init_args"]['memory_dim'],
    num_pitchers=len(meta["pitcher_to_id"]),
    num_batters=len(meta["batter_to_id"]),
    pitcher_embed_dim=32,
    batter_embed_dim=32,
    lstm_hidden_dim=128,
    num_pitch_types=4
)

# Load the trained weights
loaded_mod.load_state_dict(torch.load("./models/best_pitch_predictor_lstm.pth", map_location="cpu"))
loaded_mod.eval()


In [ ]:
probs = model.predict_next_pitch(loaded_mod, current_context=context, pitcher_id=[meta["pitcher_to_id"][506433]], 
                         batter_id=[meta["batter_to_id"][518595]], previous_pitches_in_pa=mock_memory)

In [70]:
meta['pitch_type_to_idx']
# Map the predicted probabilities to pitch type names using meta['pitch_type_to_idx']
# We'll invert the dict to get idx -> pitch_type
idx_to_pitch_type = {v: k for k, v in meta['pitch_type_to_idx'].items()}
pitch_probs = {idx_to_pitch_type[i]: float(probs[0, i]) for i in range(probs.shape[1])}
pitch_probs


{'FAST': 0.4343152940273285,
 'OFF': 0.10942773520946503,
 'BREAK': 0.4561111330986023,
 'OTH': 0.00014581962022930384}

#### original model with all savant data

In [ ]:
lstm_mod = model.train_model(train_dataset, val_dataset)

initting model architecture
starting training loop
Epoch 1, Train Loss: 0.8674, Val Loss: 0.8429
Epoch 2, Train Loss: 0.8360, Val Loss: 0.8264
Epoch 3, Train Loss: 0.8258, Val Loss: 0.8201
Epoch 4, Train Loss: 0.8204, Val Loss: 0.8119
Epoch 5, Train Loss: 0.8166, Val Loss: 0.8087
Epoch 6, Train Loss: 0.8142, Val Loss: 0.8107
Epoch 7, Train Loss: 0.8119, Val Loss: 0.8085
Epoch 8, Train Loss: 0.8102, Val Loss: 0.8089
Epoch 9, Train Loss: 0.8088, Val Loss: 0.8043
Epoch 10, Train Loss: 0.8070, Val Loss: 0.8079
Epoch 11, Train Loss: 0.8061, Val Loss: 0.8076
Epoch 12, Train Loss: 0.8050, Val Loss: 0.8034
Epoch 13, Train Loss: 0.8044, Val Loss: 0.8065
Epoch 14, Train Loss: 0.8036, Val Loss: 0.8066
Epoch 15, Train Loss: 0.8023, Val Loss: 0.8134
Epoch 16, Train Loss: 0.8020, Val Loss: 0.8092
Epoch 17, Train Loss: 0.8013, Val Loss: 0.8061
Epoch 18, Train Loss: 0.8012, Val Loss: 0.8071
Epoch 19, Train Loss: 0.8006, Val Loss: 0.8044
Epoch 20, Train Loss: 0.7997, Val Loss: 0.8083
Epoch 21, Train Lo

In [ ]:
import torch

# Load the model architecture
mock_lstm_mod = model.PitchPredictorLSTM(
    context_dim=len(context_features),
    memory_dim=len(memory_features),
    num_pitchers=len(df['pitcher'].unique()),
    num_batters=965, #len(df['batter'].unique()),
    pitcher_embed_dim=32,
    batter_embed_dim=32,
    lstm_hidden_dim=128,
    num_pitch_types=4
)

# Load the trained weights
mock_lstm_mod.load_state_dict(torch.load('best_pitch_predictor_lstm.pth', map_location='cpu'))
mock_lstm_mod.eval()

## Modified Model W/ Pitcher/Batter MLPs

In [ ]:
import pickle
import torch
import model

# Load model config/specs
with open("./models/pitch_predictor_lstm_meta1.pkl", "rb") as f:
    meta = pickle.load(f)

# Instantiate the model using the loaded specs
loaded_mod = model.PitchPredictorLSTM(
    context_dim=meta["lstm_init_args"]['context_dim'],
    memory_dim=meta["lstm_init_args"]['memory_dim'],
    num_pitchers=len(meta["pitcher_to_id"]),
    num_batters=len(meta["batter_to_id"]),
    pitcher_embed_dim=16,
    batter_embed_dim=16,
    lstm_hidden_dim=128,
    num_pitch_types=4
)

# Load the trained weights
loaded_mod.load_state_dict(torch.load("./models/pitch_predictor_lstm1.pth", map_location="cpu"))
loaded_mod.eval()


In [5]:
df = pd.read_csv('./data/final_final_encoded_savant_2024.csv')
# List of memory features (copy from your model)
memory_features = [
    'release_speed', 'release_pos_y',
    'pitch_type_group_BREAK', 'pitch_type_group_FAST', 'pitch_type_group_OFF',
    'description_ball', 'description_blocked_ball', 'description_called_strike', 'description_foul', 
    'description_foul_bunt', 'description_foul_tip', 'description_hit_by_pitch', 'description_hit_into_play', 
    'description_pitchout', 'description_swinging_strike', 'description_swinging_strike_blocked', 
    'type_B', 'type_S'
]

# Number of previous pitches to mock
num_prev_pitches = 3

# Create random data for each feature
mock_data = {}
np.random.seed(42)  # For reproducibility

# Example: assign plausible ranges for numeric features, 0/1 for one-hot/categorical
for feature in memory_features:
    if feature == 'release_speed':
        mock_data[feature] = np.random.uniform(-0.75, 100, num_prev_pitches)  # plausible pitch speeds
    elif feature == 'release_pos_y':
        mock_data[feature] = np.random.uniform(-0.7, 0.85, num_prev_pitches)   # plausible y positions
    else:
        # For one-hot/categorical features, use 0 or 1
        mock_data[feature] = np.random.randint(0, 2, num_prev_pitches).astype(float)

mock_memory = pd.DataFrame(mock_data).to_numpy()
# context_features = [
#     'stand_R', 'p_throws_R', 'balls', 'strikes',
#     'on_3b', 'on_2b', 'on_1b', 'outs_when_up', 'inning', 'inning_topbot_Top', 'at_bat_number', 'pitch_number',
#     'home_score', 'away_score', 'bat_score', 'fld_score', 'n_thruorder_pitcher', 'n_priorpa_thisgame_player_at_bat',
#     'prev_runs_scored'
# ]
context_features = [
    'stand_R', 'p_throws_R', 'balls', 'strikes', 'on_3b', 'on_2b', 'on_1b', 'outs_when_up', 'inning', 'inning_topbot_Top', 
    'at_bat_number', 'pitch_number', 'home_score', 'away_score', 'prev_runs_scored'
]
context = df.iloc[100][context_features].astype(float).values

In [6]:
# load pitcher, batter data
batter_df = pd.read_csv('./data/batting_stats_23.csv')
pitcher_df = pd.read_csv('./data/pitching_stats_23.csv')

pitcher_stat_cols = ['#days', 'Age', 'G', 'GS', 'W', 'L', 'SV', 'IP', 'H', 'R', 'ER', 'BB', 
                                  'SO', 'HR', 'HBP', 'ERA', 'AB', '2B', '3B', 'IBB', 'GDP', 'SF', 'SB', 'CS', 'PO', 'BF', 'Pit', 
                                  'Str', 'StL', 'StS', 'GB/FB', 'LD', 'PU', 'WHIP', 'BAbip', 'SO9', 'SO/W']
        
batter_stat_cols = ['#days', 'Age', 'G', 'PA', 'AB', 'R', 'H', '2B', '3B', 'HR', 'RBI', 'BB', 'IBB', 'SO', 'HBP', 
                            'SH', 'SF', 'GDP', 'SB', 'CS', 'BA', 'OBP', 'SLG', 'OPS']

# added
#print(self.pitcher_stats_df.columns)
pitcher_row = pitcher_df[pitcher_df['mlbID'] == 506433]
if not pitcher_row.empty:
    pitcher_stats_vec = torch.FloatTensor(pitcher_row[pitcher_stat_cols].values[0])
else:
    pitcher_stats_vec = torch.zeros(len(pitcher_stat_cols))

batter_row = batter_df[batter_df['mlbID'] == 518595]
if not batter_row.empty:
    batter_stats_vec = torch.FloatTensor(batter_row[batter_stat_cols].values[0])
else:
    batter_stats_vec = torch.zeros(len(batter_stat_cols))

In [ ]:
print(context.shape)
print(pitcher_stats_vec.shape)
print(batter_stats_vec.shape)

(15,)
torch.Size([37])
torch.Size([24])


In [15]:
probs = model.predict_next_pitch(loaded_mod, current_context=context, pitcher_id=[meta["pitcher_to_id"][506433]], 
                         batter_id=[meta["batter_to_id"][518595]], previous_pitches_in_pa=mock_memory,
                         pitcher_stats=pitcher_stats_vec.unsqueeze(0), batter_stats=batter_stats_vec.unsqueeze(0))
probs

array([[6.3602757e-01, 1.2465691e-01, 2.3922165e-01, 9.3892959e-05]],
      dtype=float32)

In [16]:
meta['pitch_type_to_idx']
# Map the predicted probabilities to pitch type names using meta['pitch_type_to_idx']
# We'll invert the dict to get idx -> pitch_type
idx_to_pitch_type = {v: k for k, v in meta['pitch_type_to_idx'].items()}
pitch_probs = {idx_to_pitch_type[i]: float(probs[0, i]) for i in range(probs.shape[1])}
pitch_probs

{'FAST': 0.6360275745391846,
 'OFF': 0.12465690821409225,
 'BREAK': 0.23922164738178253,
 'OTH': 9.389295883011073e-05}

## Modified with more removed data

#### pitcher game sequences

In [ ]:
import os
hasDataSet = os.path.exists('./data/train_dataset_pitcher.pkl')
print(hasDataSet)
train_dataset, val_dataset = model.load_training_data('./data/player_game_savant.csv', hasDataSet, './data/batting_stats_23.csv', './data/pitching_stats_23.csv')

True
reading batter csv
read batter csv
reading pitcher csv
read pitcher csv
loading train_dataset f from pickle
loading val_dataset f from pickle


In [ ]:
# PITCHER-GAME sequences | without description_foul_bunt and description_foul_tip + other stuff for live data model integration
lstm_mod = model.train_model(train_dataset, val_dataset)

initting model architecture
starting training loop
Epoch 1, Train Loss: 0.9004, Val Loss: 0.9194
Epoch 2, Train Loss: 0.8891, Val Loss: 0.9165
Epoch 3, Train Loss: 0.8871, Val Loss: 0.9177
Epoch 4, Train Loss: 0.8864, Val Loss: 0.9170
Epoch 5, Train Loss: 0.8855, Val Loss: 0.9129
Epoch 6, Train Loss: 0.8848, Val Loss: 0.9159
Epoch 7, Train Loss: 0.8842, Val Loss: 0.9131
Epoch 8, Train Loss: 0.8840, Val Loss: 0.9123
Epoch 9, Train Loss: 0.8837, Val Loss: 0.9189
Epoch 10, Train Loss: 0.8835, Val Loss: 0.9128
Epoch 11, Train Loss: 0.8832, Val Loss: 0.9120
Epoch 12, Train Loss: 0.8831, Val Loss: 0.9162
Epoch 13, Train Loss: 0.8831, Val Loss: 0.9146
Epoch 14, Train Loss: 0.8830, Val Loss: 0.9133
Epoch 15, Train Loss: 0.8826, Val Loss: 0.9141
Epoch 16, Train Loss: 0.8826, Val Loss: 0.9146
Epoch 17, Train Loss: 0.8826, Val Loss: 0.9133
Epoch 18, Train Loss: 0.8826, Val Loss: 0.9136
Epoch 19, Train Loss: 0.8823, Val Loss: 0.9120
Epoch 20, Train Loss: 0.8822, Val Loss: 0.9120
Epoch 21, Train Lo

#### pitcher-plate sequences

In [ ]:
import os
# without description_foul_bunt and description_foul_tip + other stuff for live data model integration
hasDataSet = os.path.exists('./data/train_dataset_f_emb.pkl')
print(hasDataSet)
train_dataset, val_dataset = model.load_training_data('./data/final_final_encoded_savant_2024.csv', hasDataSet, './data/batting_stats_23.csv', './data/pitching_stats_23.csv')

False
reading batter csv
read batter csv
reading pitcher csv
read pitcher csv
reading dataset csv
read dataset csv
processing train dataset


Processing plate appearances: 100%|██████████| 179716/179716 [02:33<00:00, 1173.10it/s]


processing val dataset


Processing plate appearances: 100%|██████████| 100815/100815 [00:23<00:00, 4266.68it/s]


finished processing datasets f


In [ ]:
# without description_foul_bunt and description_foul_tip + other stuff for live data model integration
lstm_mod = model.train_model(train_dataset, val_dataset)

initting model architecture
starting training loop
Epoch 1, Train Loss: 0.9305, Val Loss: 0.9351
Epoch 2, Train Loss: 0.9199, Val Loss: 0.9304
Epoch 3, Train Loss: 0.9178, Val Loss: 0.9328
Epoch 4, Train Loss: 0.9170, Val Loss: 0.9294
Epoch 5, Train Loss: 0.9159, Val Loss: 0.9309
Epoch 6, Train Loss: 0.9153, Val Loss: 0.9286
Epoch 7, Train Loss: 0.9148, Val Loss: 0.9280
Epoch 8, Train Loss: 0.9144, Val Loss: 0.9296
Epoch 9, Train Loss: 0.9143, Val Loss: 0.9298
Epoch 10, Train Loss: 0.9142, Val Loss: 0.9305
Epoch 11, Train Loss: 0.9140, Val Loss: 0.9304
Epoch 12, Train Loss: 0.9139, Val Loss: 0.9282
Epoch 13, Train Loss: 0.9138, Val Loss: 0.9281
Epoch 14, Train Loss: 0.9137, Val Loss: 0.9284
Epoch 15, Train Loss: 0.9137, Val Loss: 0.9271
Epoch 16, Train Loss: 0.9134, Val Loss: 0.9265
Epoch 17, Train Loss: 0.9134, Val Loss: 0.9261
Epoch 18, Train Loss: 0.9132, Val Loss: 0.9278
Epoch 19, Train Loss: 0.9130, Val Loss: 0.9299
Epoch 20, Train Loss: 0.9126, Val Loss: 0.9276
Epoch 21, Train Lo

KeyboardInterrupt: 

#### prediction

In [ ]:
# Load model config/specs
with open("./pitch_predictor_lstm_meta1.pkl", "rb") as f:
    meta = pickle.load(f)

# Instantiate the model using the loaded specs
loaded_mod = model.PitchPredictorLSTM(
    context_dim=meta["lstm_init_args"]['context_dim'],
    memory_dim=meta["lstm_init_args"]['memory_dim'],
    num_pitchers=len(meta["pitcher_to_id"]),
    num_batters=len(meta["batter_to_id"]),
    pitcher_embed_dim=16,
    batter_embed_dim=16,
    lstm_hidden_dim=128,
    num_pitch_types=4
)

# Load the trained weights
loaded_mod.load_state_dict(torch.load("./pitch_predictor_lstm1.pth", map_location="cpu"))
loaded_mod.eval()

df = pd.read_csv('./data/final_final_encoded_savant_2024.csv')
# List of memory features (copy from your model)
memory_features = [
    'release_speed', # 'release_pos_y',
    'pitch_type_group_BREAK', 'pitch_type_group_FAST', 'pitch_type_group_OFF',
    'description_ball', 'description_blocked_ball', 'description_called_strike', 'description_foul', 
    'description_hit_by_pitch', 'description_hit_into_play', #'description_foul_bunt', 'description_foul_tip', 
    'description_pitchout', 'description_swinging_strike', # 'description_swinging_strike_blocked', 
    'type_B', 'type_S'
]

# Number of previous pitches to mock
num_prev_pitches = 3

# Create random data for each feature
mock_data = {}
np.random.seed(42)  # For reproducibility

# Example: assign plausible ranges for numeric features, 0/1 for one-hot/categorical
for feature in memory_features:
    if feature == 'release_speed':
        mock_data[feature] = np.random.uniform(-0.75, 100, num_prev_pitches)  # plausible pitch speeds
    elif feature == 'release_pos_y':
        mock_data[feature] = np.random.uniform(-0.7, 0.85, num_prev_pitches)   # plausible y positions
    else:
        # For one-hot/categorical features, use 0 or 1
        mock_data[feature] = np.random.randint(0, 2, num_prev_pitches).astype(float)

mock_memory = pd.DataFrame(mock_data).to_numpy()
context_features = [
    'p_throws_R', 'balls', 'strikes', 'on_3b', 'on_2b', 'on_1b', 'outs_when_up', 'inning', 'inning_topbot_Top', 
    'at_bat_number', 'pitch_number', 'home_score', 'away_score', 'prev_runs_scored' #'stand_R', 
]
context = df.iloc[100][context_features].astype(float).values

# load pitcher, batter data
batter_df = pd.read_csv('./data/batting_stats_23.csv')
pitcher_df = pd.read_csv('./data/pitching_stats_23.csv')

pitcher_stat_cols = ['#days', 'Age', 'G', 'GS', 'W', 'L', 'SV', 'IP', 'H', 'R', 'ER', 'BB', 
                                  'SO', 'HR', 'HBP', 'ERA', 'AB', '2B', '3B', 'IBB', 'GDP', 'SF', 'SB', 'CS', 'PO', 'BF', 'Pit', 
                                  'Str', 'StL', 'StS', 'GB/FB', 'LD', 'PU', 'WHIP', 'BAbip', 'SO9', 'SO/W']
        
batter_stat_cols = ['#days', 'Age', 'G', 'PA', 'AB', 'R', 'H', '2B', '3B', 'HR', 'RBI', 'BB', 'IBB', 'SO', 'HBP', 
                            'SH', 'SF', 'GDP', 'SB', 'CS', 'BA', 'OBP', 'SLG', 'OPS']

# added
#print(self.pitcher_stats_df.columns)
pitcher_row = pitcher_df[pitcher_df['mlbID'] == 506433]
if not pitcher_row.empty:
    pitcher_stats_vec = torch.FloatTensor(pitcher_row[pitcher_stat_cols].values[0])
else:
    pitcher_stats_vec = torch.zeros(len(pitcher_stat_cols))

batter_row = batter_df[batter_df['mlbID'] == 518595]
if not batter_row.empty:
    batter_stats_vec = torch.FloatTensor(batter_row[batter_stat_cols].values[0])
else:
    batter_stats_vec = torch.zeros(len(batter_stat_cols))

In [13]:
probs = model.predict_next_pitch(loaded_mod, current_context=context, pitcher_id=[meta["pitcher_to_id"][506433]], 
                         batter_id=[meta["batter_to_id"][518595]], previous_pitches_in_pa=mock_memory,
                         pitcher_stats=pitcher_stats_vec.unsqueeze(0), batter_stats=batter_stats_vec.unsqueeze(0))
probs

array([[6.3943422e-01, 6.0463212e-02, 3.0004773e-01, 5.4788296e-05]],
      dtype=float32)

In [16]:
import model
import pickle
import torch
with open("./models/lstm_game_history_meta.pkl", "rb") as f:
    meta = pickle.load(f)
batter_df = pd.read_csv('./data/batting_stats_23.csv')
pitcher_df = pd.read_csv('./data/pitching_stats_23.csv')
with open('./data/val_dataset_pitcher.pkl', 'rb') as f:
            val_dataset = pickle.load(f)
            
pitcher_stat_cols = ['#days', 'Age', 'G', 'GS', 'W', 'L', 'SV', 'IP', 'H', 'R', 'ER', 'BB', 
                                  'SO', 'HR', 'HBP', 'ERA', 'AB', '2B', '3B', 'IBB', 'GDP', 'SF', 'SB', 'CS', 'PO', 'BF', 'Pit', 
                                  'Str', 'StL', 'StS', 'GB/FB', 'LD', 'PU', 'WHIP', 'BAbip', 'SO9', 'SO/W']
        
batter_stat_cols = ['#days', 'Age', 'G', 'PA', 'AB', 'R', 'H', '2B', '3B', 'HR', 'RBI', 'BB', 'IBB', 'SO', 'HBP', 
                            'SH', 'SF', 'GDP', 'SB', 'CS', 'BA', 'OBP', 'SLG', 'OPS']
count = 0
correct = 0
non_fast_pred_count = 0
for context, pitcher_id, batter_id, memory_seq, pitcher_stats, batter_stats, target in val_dataset:

    pitcher_row = pitcher_df[pitcher_df['mlbID'] == pitcher_id]
    if not pitcher_row.empty:
        pitcher_stats_vec = torch.FloatTensor(pitcher_row[pitcher_stat_cols].values[0])
    else:
        pitcher_stats_vec = torch.zeros(len(pitcher_stat_cols))

    batter_row = batter_df[batter_df['mlbID'] == batter_id]
    if not batter_row.empty:
        batter_stats_vec = torch.FloatTensor(batter_row[batter_stat_cols].values[0])
    else:
        batter_stats_vec = torch.zeros(len(batter_stat_cols))

    probs = model.predict_next_pitch(lstm_mod, current_context=context, pitcher_id=[pitcher_id.item()], 
                    batter_id=[batter_id.item()], previous_pitches_in_pa=memory_seq,
                    pitcher_stats=pitcher_stats_vec.unsqueeze(0), batter_stats=batter_stats_vec.unsqueeze(0))
    

    idx_to_pitch_type = {v: k for k, v in meta['pitch_type_to_idx'].items()}
    probs = {idx_to_pitch_type[i]: float(probs[0, i]) for i in range(probs.shape[1])}
    # pitch_probs

    highest_key = ''
    highest_prob = 0
    #print(probs)
    for key in probs:
        if probs[key] > highest_prob:
            highest_prob = probs[key]
            highest_key = key
    
    pitch_types = ['FAST', 'OFF', 'BREAK', 'OTH']
    print(f'Actual: {pitch_types[target]} Predicted: {highest_key} with prob {highest_prob: 0.4f}')
    count+=1
    if (highest_key != 'FAST'):
        non_fast_pred_count += 1
    if (pitch_types[target] == highest_key):
        correct += 1

In [17]:
correct/count

0.5665349355698108